# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments   Customer ShipmentStatus
0  Mar 15 2023  9:04AM  259349        10  HH-40625  Hush Hush       Released
1  Mar 15 2023  9:04AM  259349        10  HH-40626  Hush Hush       Released
2  Mar 15 2023  9:04AM  259349        10  HH-40627  Hush Hush       Released
3  Mar 15 2023  9:04AM  259349        10  HH-40628  Hush Hush       Released
4  Mar 15 2023  9:04AM  259349        10  HH-40629  Hush Hush       Released
5  Mar 15 2023  9:04AM  259349        10  HH-40630  Hush Hush       Released
6  Mar 15 2023  9:04AM  259349        10  HH-40619  Hush Hush       Released
7  Mar 15 2023  9:04AM  259349        10  HH-40620  Hush Hush       Released
8  Mar 15 2023  9:04AM  259349        10  HH-40621  Hush Hush       Released
9  Mar 15 2023  9:04AM  259349        10  HH-40622  Hush Hush       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
7   259344       Released          1
8   259347       Released          2
9   259348       Released          1
10  259349       Released         17
11  259351       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
259344                NaN       1.0
259347                NaN       2.0
259348                NaN       1.0
259349                NaN      17.0
259351                NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
259258                0.0       1.0
259311                0.0       1.0
259312                1.0       0.0
259319                1.0       0.0
259341                2.0       2.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
259258                  0         1
259311                  0         1
259312                  1         0
259319                  1         0
259341                  2         2

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               259258          0         1
1               259311          0         1
2               259312          1         0
3               259319          1         0
4               259341          2         2

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               259258                  1
1               259311                  1
2               259312         1         
3               259319         1         
4               259341         2        2

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                 Customer
0   Mar 15 2023  9:04AM  259349        10                Hush Hush
17  Mar 15 2023  9:04AM  259351        10                Hush Hush
18  Mar 15 2023  9:01AM  259348        10              Emerginnova
19  Mar 15 2023  8:59AM  259347        10         Methapharm, Inc.
20  Mar 15 2023  8:59AM  259347        10             Methapharm-G
21  Mar 15 2023  8:30AM  259319        10                Hush Hush
22  Mar 15 2023  8:28AM  259344        19  GUSA Granules USA, Inc.
23  Mar 15 2023  7:42AM  259342        19      AdvaGen Pharma, Ltd
24  Mar 15 2023  7:36AM  259341        19      AdvaGen Pharma, Ltd
28  Mar 14 2023  3:10PM  259312        12                Hush Hush

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                 Customer Executing  \
0  Mar 15 2023  9:04AM  259349        10                Hush Hush             
1  Mar 15 2023  9:04AM  259351        10                Hush Hush             
2  Mar 15 2023  9:01AM  259348        10              Emerginnova             
3  Mar 15 2023  8:59AM  259347        10         Methapharm, Inc.             
4  Mar 15 2023  8:59AM  259347        10             Methapharm-G             
5  Mar 15 2023  8:30AM  259319        10                Hush Hush         1   
6  Mar 15 2023  8:28AM  259344        19  GUSA Granules USA, Inc.             
7  Mar 15 2023  7:42AM  259342        19      AdvaGen Pharma, Ltd             
8  Mar 15 2023  7:36AM  259341        19      AdvaGen Pharma, Ltd         2   
9  Mar 14 2023  3:10PM  259312        12                Hush Hush         1   

  Released  
0       17  
1        1  
2        1  
3        2  
4        2  
5           
6        1  
7        1  
8        2  
9

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                 Customer Released  \
0  Mar 15 2023  9:04AM  259349        10                Hush Hush       17   
1  Mar 15 2023  9:04AM  259351        10                Hush Hush        1   
2  Mar 15 2023  9:01AM  259348        10              Emerginnova        1   
3  Mar 15 2023  8:59AM  259347        10         Methapharm, Inc.        2   
4  Mar 15 2023  8:59AM  259347        10             Methapharm-G        2   
5  Mar 15 2023  8:30AM  259319        10                Hush Hush            
6  Mar 15 2023  8:28AM  259344        19  GUSA Granules USA, Inc.        1   
7  Mar 15 2023  7:42AM  259342        19      AdvaGen Pharma, Ltd        1   
8  Mar 15 2023  7:36AM  259341        19      AdvaGen Pharma, Ltd        2   
9  Mar 14 2023  3:10PM  259312        12                Hush Hush            

  Executing  
0            
1            
2            
3            
4            
5         1  
6            
7            
8         2  
9         1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse          Customer Released Executing
0  Mar 15 2023  9:04AM  259349        10         Hush Hush       17          
1  Mar 15 2023  9:04AM  259351        10         Hush Hush        1          
2  Mar 15 2023  9:01AM  259348        10       Emerginnova        1          
3  Mar 15 2023  8:59AM  259347        10  Methapharm, Inc.        2          
4  Mar 15 2023  8:59AM  259347        10      Methapharm-G        2

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse          Customer  Released  \
0  Mar 15 2023  9:04AM  259349        10         Hush Hush      17.0   
1  Mar 15 2023  9:04AM  259351        10         Hush Hush       1.0   
2  Mar 15 2023  9:01AM  259348        10       Emerginnova       1.0   
3  Mar 15 2023  8:59AM  259347        10  Methapharm, Inc.       2.0   
4  Mar 15 2023  8:59AM  259347        10      Methapharm-G       2.0   

   Executing  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse          Customer  Released  \
0  Mar 15 2023  9:04AM  259349        10         Hush Hush      17.0   
1  Mar 15 2023  9:04AM  259351        10         Hush Hush       1.0   
2  Mar 15 2023  9:01AM  259348        10       Emerginnova       1.0   
3  Mar 15 2023  8:59AM  259347        10  Methapharm, Inc.       2.0   
4  Mar 15 2023  8:59AM  259347        10      Methapharm-G       2.0   

   Executing  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         1556061      23.0        1.0
12          259312       0.0        1.0
19         1296596       6.0        2.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  1556061      23.0        1.0
1        12   259312       0.0        1.0
2        19  1296596       6.0        2.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10      23.0        1.0
1        12       0.0        1.0
2        19       6.0        2.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released   23.0
1        12   Released    0.0
2        19   Released    6.0
3        10  Executing    1.0
4        12  Executing    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12   19
Status                   
Executing   1.0  1.0  2.0
Released   23.0  0.0  6.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12   19
0          Executing   1.0  1.0  2.0
1           Released  23.0  0.0  6.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12   19
0  Executing   1.0  1.0  2.0
1   Released  23.0  0.0  6.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()